In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import bernoulli,norm,uniform,rankdata
import tqdm
import sys

In [ ]:
# CI function
def CI(data,alpha):
    sample_mean=np.mean(data) # data is a list!
    sample_sigma=np.std(data)
    critical_point = norm.ppf(1-alpha/2)
    LB=sample_mean-critical_point*sample_sigma/np.sqrt(len(data))
    UB=sample_mean+critical_point*sample_sigma/np.sqrt(len(data))
    return round(LB,3),round(UB,3)

In [ ]:
def tournament(simulation_times,contestants,sigma0_RMS,range_RMS):
    N=simulation_times
    MC=np.zeros(N)
    NC=contestants # Number of contestants
    Prob_list=np.arange(0.05,1,0.1)
    for k in tqdm.tqdm(range(N)):
        Q=np.zeros((10,10))
        Ber=np.zeros((10,10))
        sigma_RMS=np.zeros(NC)
        score=np.zeros(NC)
        determine_matrix=np.zeros((10,10))

        for i in range(Q.shape[1]): # Create bernouli result matrix
            p=Prob_list[i]
            Q[:,i]=p
            for j in range(Q.shape[0]):
                Ber[j,i]=bernoulli.rvs(p, size=1) 
            ## We can't merge this loop into next one because everyone has the same Ber. result!

        for i in range(NC): # for each contestant
            sigma_RMS[i]=sigma0_RMS+range_RMS*i/NC
            prediction=np.zeros((Q.shape[0],Q.shape[1]))
            for a in range(Q.shape[0]): # Create indicator to determine plus or minus sigma
                for b in range(Q.shape[1]):
                    Z=np.random.normal(0,1)
                    if Z>0:
                        prediction[a,b] = Q[a,b]+sigma_RMS[i]
                    else:
                        prediction[a,b] = Q[a,b]-sigma_RMS[i]
            score[i] = np.sum((Ber-prediction)**2)
        MC[k]=np.where(score==np.min(score))[0][0]
    print('\n Average rank of the winner by Monte Carlo is: {}'.format(np.mean(MC)))
    print('\n The confidence interval is: {}'.format(CI(MC,0.05)))
    return MC

In [ ]:
def plotfig(MC,contestants,scenario_label,count):
#     print('\n The winner is Rank {} with the lowest score = {}'.format(np.where(score==np.min(score))[0][0],round(np.min(score),2)))
#     plt.figure(figsize=(10,5))
#     plt.plot(score,linewidth=2)
#     plt.title('Score for each contestant',fontsize=13)
#     plt.xlabel('Rank of each contestant',fontsize = 10)
#     plt.ylabel('Score',fontsize = 10)
#     plt.xticks(fontsize=10);
#     plt.yticks(fontsize=10)
#     plt.grid( which = 'major' )
#     plt.grid( which = 'minor', linestyle = ':' )
#     plt.savefig('Plot {}-{}.pdf'.format(scenario_label,count))
    
    plt.figure(figsize=(7,5))
    plt.hist(MC,bins=30)
    plt.title('Winner distribution',fontsize=13)
    plt.xlabel('Rank of contestants',fontsize = 12)
    plt.ylabel('Frequency',fontsize = 12)
    plt.xticks(fontsize=12);
    plt.yticks(fontsize=12)
    plt.xlim(0,contestants)
    plt.savefig('Hist {}-{}.pdf'.format(scenario_label,count))
    plt.show()

In [ ]:
'''
Scenario1: contestants = [300,100,500]
Scenario2: sigma0_RMS = [0,0.05,0.1] 
Scenario3: range_RMS = [0.3,0.1,0.5] 
Scenario4: probability
'''
contestants=[300,100,500]
sigma0_RMS=[0,0.05,0.1]
range_RMS=[0.3,0.1,0.5]

simulation_times=100 ##

In [ ]:
'''
Adjust Scenario 1, Control others
'''
count=1
j,k=0,0
sys.stdout = open("Details of scenario 1.txt", "w")
for i in range(3):
    print('*************************************************************')
    print('Scenario 1-{}: {} contestants, Sigma0 = {} with length {}'.format(count,contestants[i],sigma0_RMS[j],range_RMS[k]))
    print('*************************************************************')
    MC=tournament(simulation_times,contestants[i],sigma0_RMS[j],range_RMS[k])
    plotfig(MC,contestants[i],1,count)
    print('\n')
    count+=1

In [ ]:
'''
Adjust Scenario 2, Control others
'''
count=1
i,k=0,0
sys.stdout = open("Details of scenario 2.txt", "w")
for j in range(3):
    print('*************************************************************')
    print('Scenario 2-{}: {} contestants, Sigma0 = {} with length {}'.format(count,contestants[i],sigma0_RMS[j],range_RMS[k]))
    print('*************************************************************')
    MC=tournament(simulation_times,contestants[i],sigma0_RMS[j],range_RMS[k])
    plotfig(MC,contestants[i],2,count)
    print('\n')
    count+=1

In [ ]:
'''
Adjust Scenario 3, Control others
'''
count=1
i,j=0,0
sys.stdout = open("Details of scenario 3.txt", "w")
for k in range(3):
    print('*************************************************************')
    print('Scenario 3-{}: {} contestants, Sigma0 = {} with length {}'.format(count,contestants[i],sigma0_RMS[j],range_RMS[k]))
    print('*************************************************************')
    MC=tournament(simulation_times,contestants[i],sigma0_RMS[j],range_RMS[k])
    plotfig(MC,contestants[i],3,count)
    print('\n')
    count+=1

In [ ]:
'''
Adjust Scenario 4, Control others
'''
def tournament_prob(simulation_times,contestants,sigma0_RMS,range_RMS,prob_start,prob_end,prob_tick):
    N=simulation_times
    MC=np.zeros(N)
    NC=contestants # Number of contestants
    Prob_list=np.arange(prob_start,prob_end,prob_tick)
    for k in range(N):
        Q=np.zeros((10,len(Prob_list)))
        Ber=np.zeros((10,len(Prob_list)))
        sigma_RMS=np.zeros(NC)
        score=np.zeros(NC)
        determine_matrix=np.zeros((10,len(Prob_list)))

        for i in range(Q.shape[1]): # Create bernouli result matrix
            p=Prob_list[i]
            Q[:,i]=p
            for j in range(Q.shape[0]):
                Ber[j,i]=bernoulli.rvs(p, size=1) 
            ## We can't merge this loop into next one because everyone has the same Ber. result!

        for i in range(NC): # for each contestant
            sigma_RMS[i]=sigma0_RMS+range_RMS*i/NC
            prediction=np.zeros((Q.shape[0],Q.shape[1]))
            for a in range(Q.shape[0]): # Create indicator to determine plus or minus sigma
                for b in range(Q.shape[1]):
                    Z=np.random.normal(0,1)
                    if Z>0:
                        prediction[a,b] = Q[a,b]+sigma_RMS[i]
                    else:
                        prediction[a,b] = Q[a,b]-sigma_RMS[i]
            score[i] = np.sum((Ber-prediction)**2)
        MC[k]=np.where(score==np.min(score))[0][0]
    print('\n Average rank of the winner by Monte Carlo is: {}'.format(np.mean(MC)))
    print('\n The confidence interval is: {}'.format(CI(MC,0.05)))
    return MC

In [ ]:
prob_start=[0.05,0.01,0.05]
prob_end=[1,1,1]
prob_tick=[0.1,0.01,0.05]
count=1
i,j,k=0,0,0
sys.stdout = open("Details of scenario 4.txt", "w")
for m in range(3):
    print('*************************************************************')
    print('Scenario 4-{}: Change probability distribution from {} to {} with {}'.format(count,prob_start[m],prob_end[m]-prob_tick[m],prob_tick[m]))
    print('*************************************************************')
    MC=tournament_prob(simulation_times,contestants[i],sigma0_RMS[j],range_RMS[k],prob_start[m],prob_end[m],prob_tick[m])
    plotfig(MC,contestants[i],4,count)
    print('\n')
    count+=1

In [ ]:
'''
Create all scenarios

count=1
for i in range(3):
    for j in range(3):
        for k in range(3):
            print('*************************************************************')
            print('Scenario {}: {} contestants, Sigma0 = {} with length {}'.format(count,contestants[i],sigma0_RMS[j],range_RMS[k]))
            print('*************************************************************')
            MC,score=tournament(simulation_times,contestants[i],sigma0_RMS[j],range_RMS[k])
            plotfig(score,count)
            count+=1